In [ ]:
import os

%cd /content
if not os.path.exists('MODNet'):
  !git clone https://github.com/ZHKKKe/MODNet
%cd MODNet/
pretrained_ckpt = 'pretrained/modnet_photographic_portrait_matting.ckpt'
if not os.path.exists(pretrained_ckpt):
  !gdown --id 1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz \
          -O pretrained/modnet_photographic_portrait_matting.ckpt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
from google.colab import files

input_folder = 'demo/image_matting/colab/input'
if os.path.exists(input_folder):
  shutil.rmtree(input_folder)
os.makedirs(input_folder)

output_folder = 'demo/image_matting/colab/output'
if os.path.exists(output_folder):
  shutil.rmtree(output_folder)
os.makedirs(output_folder)

foreground_folder = 'demo/image_matting/colab/foreground'
if os.path.exists(foreground_folder):
  shutil.rmtree(foreground_folder)
os.makedirs(foreground_folder)

# upload images (PNG or JPG)
image_names = list(files.upload().keys())
for image_name in image_names:
  shutil.move(image_name, os.path.join(input_folder, image_name))


In [ ]:
!python -m demo.image_matting.colab.inference \
        --input-path demo/image_matting/colab/input \
        --output-path demo/image_matting/colab/output \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt


In [ ]:
import numpy as np
from PIL import Image
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from skimage import io
def combined_display(image, matte):
  w, h = image.width, image.height
  rw, rh = 800, int(h * 800 / (3 * w))
  image = np.asarray(image)
  if len(image.shape) == 2:
    image = image[:, :, None]
  if image.shape[2] == 1:
    image = np.repeat(image, 3, axis=2)
  elif image.shape[2] == 4:
    image = image[:, :, 0:3]
  matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2) / 255
  foreground = image * matte + np.full(image.shape, 255) * (1 - matte)
  
  combined = np.concatenate((image, foreground, matte * 255), axis=1)
  foreground = Image.fromarray(np.uint8(foreground)).resize((w, h))
  return foreground

from google.colab import files as FILE
def notcombined_display(image, matte):

  w, h = image.width, image.height
  rw, rh = 800, int(h * 800 / (3 * w))

  image = np.asarray(image)
  if len(image.shape) == 2:
    image = image[:, :, None]
  if image.shape[2] == 1:
    image = np.repeat(image, 3, axis=2)
  elif image.shape[2] == 4:
    image = image[:, :, 0:3]
  matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2) / 255
  foreground = image * matte + np.full(image.shape, 255) * (1 - matte)
  
  combined = np.concatenate((image, foreground, matte * 255), axis=1)
  combined = Image.fromarray(np.uint8(combined)).resize((rw, rh))
  return image,matte,foreground
  
image_names = os.listdir(input_folder)
for image_name in image_names:
  matte_name = image_name.split('.')[0] + '.png'
  image = Image.open(os.path.join(input_folder, image_name))
  matte = Image.open(os.path.join(output_folder, matte_name))
  matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2) / 255
  bitnd = image*matte+np.full(np.asarray(image).shape, 255) * (1 - matte)
  print(bitnd.shape)
  bitnd = Image.fromarray(np.uint8(bitnd))
  bitnd.save(f'demo/image_matting/colab/foreground/{image_name}')
  print(image_name, '\n')



In [ ]:
os.getcwd()

In [ ]:
os.listdir()

In [ ]:
os.chdir('demo/image_matting/colab/')

In [ ]:
os.listdir()

In [ ]:
if os.path.exists("/content/drive/My Drive/foreground"):
  shutil.rmtree("/content/drive/My Drive/foreground")
!cp  -r 'foreground/' "/content/drive/My Drive/"

In [ ]:
zip_filename = 'matte.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)

os.system(f"zip -r -j {zip_filename} {foreground_folder}/*")
files.download(zip_filename)

In [ ]:
zip= 'matte_trimap.zip'
if os.path.exists(zip):
  os.remove(zip)

os.system(f"zip -r -j {zip} demo/image_matting/colab/output/*")
files.download(zip)